# Read me

Hello, This is for begginers who want to know how you can handle this dataset which consists of some csv file and image datasaet.

Traditionally, a large dataset needs to be transoformed with TF but, if you don't know how it works, this notebook is of use to you.

This notebook is just for handling data such as loading the dataset, stacking dataset, and analyzing with deep neural network (especially, CNN).

It will be helpful for preparing analysis to someone who wants to submit.

If you have any questions, please leave the comments.


## **Knowledge can be improved by being shared.**

Please upvote!!


## [You can learn more skills for handling dataset or neural network.]

### [Advanced parallel combination DNN with CNN] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/improved-dnn-cnn-models-for-beginners
 
### [Image data handling without memory exploded] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners

### [Data handling & Deep learning] - Titanic competition (best score!!)
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
### [Deep learning model with SeLU activation function] - Titanic competition
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

### [Preparing a completed dataset with proper imputation method] - Titanic competition
 - https://www.kaggle.com/pythonash/making-completed-dataset

**Let's start!**

# Import some libraries for handling dataset.

In [ ]:
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
# seaborn is of use for visualizing.
import seaborn as sns

# load train, test, and submission sample dataset.
train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

## Glancing the train csv dataset.

All columns contain binary values except for 'Id' and 'Pawpularity'

In [ ]:
train_csv

Wow, there is no null-value.

So, we don't have to mind about imputation.

In [ ]:
train_csv.isnull().sum()

Furthermore, there is no duplicated data.

In [ ]:
train_csv.drop_duplicates()

## Count the binary values.

All data is unbalanced, but you don't have to worry about that.

I will expalin the reason why it is okay in next notebook (coming soon).

In [ ]:
for i in train_csv.drop(['Id','Pawpularity'],axis=1):
    sns.countplot(train_csv[i])
    plt.show()

The target variable of train dataset is distributed as below.

In this part, you have to determine whether you truncate some outliers or not.

However, I recommend that you don't truncate because I think those are a part of dataset, too.

We don't know what the truncated data affects in using deep learning.

It's just on my experience so, you can select and it's all up to you!

In [ ]:
sns.distplot(train_csv['Pawpularity'])

## Glancing the test dataset.

In [ ]:
test_csv

It has not null-values, either.

In [ ]:
test_csv.isnull().sum()

## Finally, Let's identify the submission form.

In [ ]:
submission

# Handling image dataset.

In [ ]:
# Set the path for loading image dataset.
os.chdir('../input/petfinder-pawpularity-score/train')

# We can find the size of each image data from this procedure
size_data = pd.DataFrame()
for file in os.listdir():
    imgg = cv2.imread(file)
    w,h,c = imgg.shape
    size_data=size_data.append([[w,h,c,imgg.size/3]])
size_data

## What is the minimum size?

In [ ]:
size_data[size_data[3] == size_data[3].min()]

## What is the number of most things?

In [ ]:
size_data[3].value_counts()

## Checking the pixel structure of that.

- It is derived from 960 * 720.

In [ ]:
size_data[size_data[3] == 691200]

## Load train image dataset and rescaling.

This part is crucial for analysis and is devided into 3 parts.

1. Loading the image dataset.

2. Changing the shape of each image into 64 * 64.

- Why 64 * 64 ?? the reason is that a large image dataset need many memories so, if you change your image pixel as small thing, memory allocation will be more free.

3. Rescaling the pixels by deviding with 255.

In this procedure, the parameter, cv2.INTER_AREA, is useful for interpolation.

There are several interpolation methods, but I recommend this.

In [ ]:
train_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    train_img.append(file/255)
train_img[:5]

## For corresponding the image with each id.

Each image data has to correspond to its own id.

In [ ]:
train_img_name = []
for i in os.listdir():
    train_img_name.append(i)
train_img_name[:5]

## Checking the file name.

Before we match the image data with its own id, we have to check the file name whether file name has identical rule or not.

If a file name has not '.jpg', it will be shown.

In [ ]:
for name in train_img_name:
    if name[-4:] != '.jpg':
        print(name)

## Matching the image dataset order with csv file order.

In [ ]:
train_csv_data = pd.DataFrame()
for img, name in zip(train_img, train_img_name):
    name=name[:-4]
    location = train_csv[train_csv['Id'] == name].index[0]
    train_csv_data= train_csv_data.append([train_csv.loc[location]])
train_csv_data

Reindexing the train csv data.

In [ ]:
train_csv_data=train_csv_data.reset_index().drop(['index'],axis=1)
train_csv_data

## Checking the resized and rescaled image with original image data.

It's original data.

In [ ]:
image_1 = cv2.imread('./'+train_csv_data['Id'][0]+'.jpg')
plt.imshow(image_1)

It's resized and rescaled data.

In [ ]:
plt.imshow(train_img[0])

It's original data.

In [ ]:
image_2 = cv2.imread('./'+train_csv_data['Id'][1]+'.jpg')
plt.imshow(image_2)

It's resized and rescaled data.

In [ ]:
plt.imshow(train_img[1])

## Load test image dataset and rescaling

What is the shape of test dataset??

It's (128, 128, 3), but we have to transforme this simze into (64, 64, 3) because of memory allocation.

For analyzing the test dataset, we will match the sizes of both train image and test image, identically.

In [ ]:
os.chdir('../test')

for i in os.listdir():
    file = cv2.imread(i)
    print(file.shape)

test dataset resize

In [ ]:
test_img = []
for i in os.listdir():
    file = cv2.imread(i)
    file=cv2.resize(file,(64,64), interpolation=cv2.INTER_AREA)
    test_img.append(file/255)
test_img[:5]

Below procedure is identical with train dataset.

In [ ]:
test_img_name = []
for i in os.listdir():
    test_img_name.append(i)
test_img_name[:5]

In [ ]:
test_csv_data = pd.DataFrame()
for img, name in zip(test_img, test_img_name):
    name=name[:-4]
    location = test_csv[test_csv['Id'] == name].index[0]
    test_csv_data= test_csv_data.append([test_csv.loc[location]])
test_csv_data=test_csv_data.reset_index().drop(['index'],axis=1)
test_csv_data

In [ ]:
test_1 = cv2.imread('./'+test_csv_data['Id'][0]+'.jpg')
plt.imshow(test_1)

In [ ]:
plt.imshow(test_img[0])

## Prepare the train input, target and test input.

In [ ]:
train_csv_x = train_csv_data.drop(['Id','Pawpularity'],axis=1)
train_y = train_csv_data['Pawpularity']

test_csv_x = test_csv_data.drop(['Id'],axis=1)

# It has done! 

You are ready to use this dataset as train and test for getting score.

Next step is preparing your submission.

# Analyzing with neural network.


We will use a special deep learning structure.

These file is devided into csv file and image file.

So, It is proper approach to use two input layers.

How can it work??

Let's start!

## Set your model

In this part, please read the codes carefully, we will use two input layers.

In [ ]:
# ##################### CSV FILE INPUT & IMG FILE INPUT LAYER ###########################
# csv_input = tf.keras.Input(shape = train_csv_x.shape[1:], name = 'CSV_Input')        ##
# img_input = tf.keras.Input(shape = np.array(train_img).shape[1:], name = 'IMG_Input')##
# #######################################################################################
#                                         ##
#                                         ##
#                                         ##
# ##################### CSV FILE HIDDEN LAYER STRUCTURE  ######################################
# csv_hidden1 = tf.keras.layers.Dense(200, activation='relu', name='CSV_Hidden1')(csv_input)   ##
# csv_hidden2 = tf.keras.layers.Dense(200, activation='relu', name='CSV_Hidden2')(csv_hidden1)##
# csv_hidden3 = tf.keras.layers.Dense(200, activation='relu', name='CSV_Hidden3')(csv_hidden2)##
# csv_dropout = tf.keras.layers.Dropout(0.5, name ='CSV_Dropout')(csv_hidden3)               ##
# csv_hidden4 = tf.keras.layers.Dense(200, activation='relu', name='CSV_Hidden4')(csv_dropout)##
# csv_hidden5 = tf.keras.layers.Dense(200, activation='relu', name='CSV_Hidden5')(csv_hidden4)##
# #############################################################################################
#                                          #
#                                          #
#                                          #
# ##################### IMG FILE CONVOLUTIONAL LAYER STRUCTURE  ######################################
# img_conv1 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 5, strides= 1, padding = 'same',   ##
#                                    activation = 'relu', name='IMG_Conv1')(img_input)              ##
# img_pool1 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool1')(img_conv1)                           ##
# img_conv2 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 4, strides= 1, padding = 'same',  ##
#                                    activation = 'relu', name='IMG_Conv2')(img_pool1)              ##
# img_conv3 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 4, strides =1, padding = 'same',  ##
#                                    activation = 'relu', name='IMG_Conv3')(img_conv2)              ##
# img_pool2 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool2')(img_conv3)                           ##
# img_conv4 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 3, strides =1, padding = 'same',  ##
#                                    activation = 'relu', name='IMG_Conv4')(img_pool2)              ##
# img_pool3 = tf.keras.layers.MaxPool2D(3, name = 'IMG_Pool3')(img_conv4)                           ##
# img_conv5 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 3, strides =1, padding = 'same',  ##
#                                    activation = 'relu', name='IMG_Conv5')(img_pool3)              ##
# img_conv6 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 3, strides =1, padding = 'same',  ##
#                                    activation = 'relu', name='IMG_Conv6')(img_conv5)              ##
# img_pool4 = tf.keras.layers.MaxPool2D(2, name = 'IMG_Pool4')(img_conv6)                           ##
# img_flatten = tf.keras.layers.Flatten(name = 'IMG_Flatten')(img_pool4)                            ##
# img_dense1 = tf.keras.layers.Dense(300, activation = 'relu', name='IMG_Dense1')(img_flatten)     ##
# img_dropout1 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout1')(img_dense1)                      ##
# img_dense2 = tf.keras.layers.Dense(300, activation = 'relu', name='IMG_Dense2')(img_dropout1)    ##
# img_dropout2 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout2')(img_dense2)                      ##
# ####################################################################################################
#                                         ##
#                                         ##
#                                         ##
# ##################### CSV FILE INPUT & IMG FILE OUTPUT LAYER ############
# csv_output = tf.keras.layers.Dense(1, name = 'CSV_Output')(csv_hidden5)##
# img_output = tf.keras.layers.Dense(1,name = 'IMG_Output')(img_dropout2)##
# #########################################################################
#                                         ##
#                                         ##
#                                         ##
# ############################################# MODEL SETTING  ####################################################
# model = tf.keras.Model(inputs=[csv_input, img_input], outputs=[csv_output, img_output], name='Pythonash_model')##
# #################################################################################################################

In [ ]:
csv_input = tf.keras.Input(shape = train_csv_x.shape[1:], name = 'CSV_Input')
img_input = tf.keras.Input(shape = np.array(train_img).shape[1:], name = 'IMG_Input')

csv_hidden1 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden1')(csv_input)
csv_hidden2 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden2')(csv_hidden1)
csv_hidden3 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden3')(csv_hidden2)
csv_hidden4 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden4')(csv_hidden3)
csv_hidden5 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden5')(csv_hidden4)
csv_hidden6 = tf.keras.layers.Dense(200, activation='elu', kernel_initializer = 'he_normal', name='CSV_Hidden6')(csv_hidden5)
csv_dropout = tf.keras.layers.Dropout(0.5, name ='CSV_Dropout')(csv_hidden6)

img_conv1 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv1')(img_input)
img_conv2 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv2')(img_conv1)
img_pooling1 = tf.keras.layers.MaxPooling2D(4, name= 'IMG_Max1')(img_conv2)

img_conv3 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv3')(img_pooling1)
img_conv4 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv4')(img_conv3)
img_pooling2 = tf.keras.layers.MaxPooling2D(4, name= 'IMG_Max2')(img_conv4)

img_conv5 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv5')(img_pooling2)
img_conv6 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv6')(img_conv5)
img_pooling3 = tf.keras.layers.MaxPooling2D(3, name= 'IMG_Max3')(img_conv6)

img_dropout = tf.keras.layers.Dropout(0.5, name = 'IMG_Dropout')(img_pooling3)
img_conv7 = tf.keras.layers.Conv2D(120, 4, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal',name='IMG_Conv7')(img_dropout)

img_hidden1 = tf.keras.layers.Dense(300, activation='elu', kernel_initializer = 'he_normal',name='IMG_hidden1')(img_conv7)
img_dropout1 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout1')(img_hidden1)

img_hidden2 = tf.keras.layers.Dense(600, activation='elu', kernel_initializer = 'he_normal',name='IMG_hidden2')(img_dropout1)

img_gpool = tf.keras.layers.GlobalAvgPool2D(name = 'IMG_Gpool')(img_hidden2)

img_dropout2 = tf.keras.layers.Dropout(0.5, name='IMG_Dropout2')(img_gpool)



csv_output = tf.keras.layers.Dense(1, name = 'CSV_Output')(csv_dropout)
img_output = tf.keras.layers.Dense(1,name = 'IMG_Output')(img_dropout2)

model = tf.keras.Model(inputs=[csv_input, img_input], outputs=[csv_output, img_output], name='Pythonash_model')

## model summary

It shows that your model structure, simply.

In [ ]:
model.summary()

## plot your model.

This plot shows your model and you can figure out you model structure, intuitively.

In [ ]:
# move you current directory to back.
os.chdir('../')
os.chdir('../')
os.chdir('../')
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

## Compile and fit your model.

In [ ]:
learning_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate = 0.002,
decay_steps = 10000,
decay_rate = 0.99)

opt = tf.keras.optimizers.Adam(learning_rate = learning_schedule)
model.compile(loss=['mse','mse'], loss_weights=[0.5, 0.5], optimizer = opt, metrics = tf.keras.metrics.RootMeanSquaredError())

epoch_number = 20

check_1 = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only=True, verbose=2)
# check_2 = tf.keras.callbacks.EarlyStopping(patience = epoch_number * 0.1, monitoring ='val_loss', verbose=2,
#                                           restore_best_weight=True)

In [ ]:
model.fit( 
    x= [train_csv_x, np.array(train_img)], y = [train_y, train_y], epochs=epoch_number, 
    validation_split=0.2, verbose =2, workers=3, batch_size = 100, validation_batch_size = 100,
    callbacks = [check_1])

In [ ]:
best_model = tf.keras.models.load_model('pythonash_model.h5')
csv_result, img_result = best_model.predict([test_csv_x, np.array(test_img)])
final_result = pd.DataFrame(0.5 * csv_result + 0.5 * img_result)
final_result.columns =['Pawpularity']
final_result

In [ ]:
for ids, paw in zip(test_csv_data['Id'], final_result['Pawpularity']):
    location = submission[submission['Id'] == ids].index[0]
    submission['Pawpularity'].loc[location] = paw
submission
submission.to_csv('./working/submission.csv',index=False)



# It's your turn!!

You have many opportunities that you can change this model parameters and get your submission score.

I recommend that you change the hyper parameters such as learning_rate, batch_size, activation function, the number of neurons, layers, and so on...

If you get any helps from my notebook, please upvote!!

Fingers crossed!!